In [72]:
import pandas as pd

Importing the Data from file storage and performing the basic checks

In [73]:
df=pd.read_csv("data/gemstone.csv")

In [74]:
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


Checking Null and Duplicate Values

In [75]:
df.isnull().sum()

id         0
carat      0
cut        0
color      0
clarity    0
depth      0
table      0
x          0
y          0
z          0
price      0
dtype: int64

In [76]:
df.duplicated().sum()

0

Dropping the unneccessary columns

In [77]:
df=df.drop(['id'],axis=1)

Spliting the Independent and Dependent features . We need to divide features into Dataframe type not series

In [78]:
X=df.iloc[:,:-1]
Y=pd.DataFrame(df.iloc[:,-1])

In [79]:
# Other way of spliting Independent and Dependent Features

# X=df.drop(['price',axis=1])
#Y=df[['price']]

In [80]:
X.head()

,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77


In [81]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193573 entries, 0 to 193572
Data columns (total 9 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   carat    193573 non-null  float64
 1   cut      193573 non-null  object 
 2   color    193573 non-null  object 
 3   clarity  193573 non-null  object 
 4   depth    193573 non-null  float64
 5   table    193573 non-null  float64
 6   x        193573 non-null  float64
 7   y        193573 non-null  float64
 8   z        193573 non-null  float64
dtypes: float64(6), object(3)
memory usage: 13.3+ MB


In [82]:
Y.head()

,price
0,13619
1,13387
2,2772
3,666
4,14453


Separating Categorical and Numerical features of Independent Features as we do not make changes to the Output feature. We need to keep dependent features as is it.

In [83]:
numerical_features=X.columns[X.dtypes !='object']
categorical_features=X.columns[X.dtypes =='object']
print(numerical_features)
print(categorical_features)

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')
Index(['cut', 'color', 'clarity'], dtype='object')


In [84]:
# Other way of separating Categorical and Numerical features
# categorical_cols = X.select_dtypes(include='object').columns
# numerical_cols = X.select_dtypes(exclude='object').columns

Creating a list of categories in a sequence according to their ranks

In [85]:
print(df.cut.unique())
print(df.color.unique())
print(df.clarity.unique())

['Premium' 'Very Good' 'Ideal' 'Good' 'Fair']
['F' 'J' 'G' 'E' 'D' 'H' 'I']
['VS2' 'SI2' 'VS1' 'SI1' 'IF' 'VVS2' 'VVS1' 'I1']


For e.g in cut categories the "Ideal" has a rank of 5 which is highest rank. For this case study Highest the index is in category highest the rank it is.

In [86]:
cut_categories=['Fair','Good','Very Good','Premium','Ideal']
color_categories=['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories=['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

Importing all the required Packages and Modules for Pipeline creation

In [87]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

Creating the separate Pipelines for categorical and numerical columns each and using the Columns Transformer for applying the Pipelines by criteria

In [88]:
numerical_pipeline=Pipeline(
    steps=[
        ('Imputer',SimpleImputer(strategy='mean')),
        ('Scaler',StandardScaler()),
        ]
)

categorical_pipeline=Pipeline(
    steps=[
        ('Imputer',SimpleImputer(strategy='most_frequent')),
        ('Ordinal',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ('Scaler',StandardScaler())
        ]
)

processor=ColumnTransformer([
    ('num_pipe',numerical_pipeline,numerical_features),
    ('category_pipe',categorical_pipeline,categorical_features)
])

Performing the Train test split and applying the Pipeline on X_train and X_test data.

In [89]:
from sklearn.model_selection  import train_test_split

In [90]:
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=42)

In [91]:
X_train_scaled =processor.fit_transform(X_train)
X_test_scaled =processor.transform(X_test)

Importing the required Regression models and training them on Train data

In [92]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet

In [93]:
Linear=LinearRegression()
Lasso=Lasso()
Ridge=Ridge()
ElasticNet=ElasticNet()

In [94]:
Linear.fit(X_train_scaled, y_train)

LinearRegression()

In [95]:
Lasso.fit(X_train_scaled, y_train)

Lasso()

In [96]:
Ridge.fit(X_train_scaled, y_train)

Ridge()

In [97]:
ElasticNet.fit(X_train_scaled,y_train)

ElasticNet()

In [98]:
#Linear_predictions = Linear.predict(X_test_scaled)
#Lasso_predictions = Lasso.predict(X_test_scaled)
#Ridge_predictions = Ridge.predict(X_test_scaled)

Checking the Trained model and it performance

In [99]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score

In [105]:
models=[Linear,Lasso,Ridge,ElasticNet]
for i in models:
    y_pred=i.predict(X_test_scaled)
    mse=mean_squared_error(y_test,y_pred)
    mae=mean_absolute_error(y_test,y_pred)
    r2score=r2_score(y_test,y_pred)
    adjusted_r2score =1-(((1-r2score)*(len(X_test)-1))/(len(X_test)-len(X.columns)-1))
    print(f"The Perfomance of the {str(i)} model is as follows:",end='\n')
    print(f"MSE : {mse}")
    print(f"MAE : {mae}")
    print(f"R2 score : {r2score}")
    print(f"Adjusted R2 score : {adjusted_r2score}",end='\n\n\n')

The Perfomance of the LinearRegression() model is as follows:
MSE : 1029473.3531156846
MAE : 675.0758270067483
R2 score : 0.9362906819996049
Adjusted R2 score : 0.936280806627382


The Perfomance of the Lasso() model is as follows:
MSE : 1029533.1506505491
MAE : 676.2421173665508
R2 score : 0.9362869814082755
Adjusted R2 score : 0.9362771054624361


The Perfomance of the Ridge() model is as follows:
MSE : 1029482.8101268952
MAE : 675.1077629781329
R2 score : 0.9362900967491632
Adjusted R2 score : 0.9362802212862226


The Perfomance of the ElasticNet() model is as follows:
MSE : 2351174.8713978743
MAE : 1060.9432977143008
R2 score : 0.8544967219374031
Adjusted R2 score : 0.8544741679519641


